# Pandas Express
# TTC-Delays project
Team Members: 

Gaspare Castagna

Amirhossein Heidari

Avery Hoffer

Michael Metallo

In [ ]:
import numpy as numpy
import pandas as pd
import os
import numpy as np
#!pip install rapidfuzz
import re
import datetime as dt
import glob


Import, read, concat bus delay data

In [2]:
def read_excel_sheets(file_path):
    xls = pd.ExcelFile(file_path)
    data_frames = []
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name)
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

In [3]:
df_2014 = read_excel_sheets('bus_2014.xlsx')
df_2015 = read_excel_sheets('bus_2015.xlsx') 
df_2016 = read_excel_sheets('bus_2016.xlsx') 
df_2017 = read_excel_sheets('bus_2017.xlsx') 


In [4]:
df_2018 = read_excel_sheets('bus_2018.xlsx')
df_2018.columns = list(df_2018.columns[:-1]) + ['Min Delay2']


In [5]:
df_2018['min_delay_combined'] = df_2018['Min Delay'].combine_first(df_2018['Min Delay2'])
df_2018.drop(columns=['Min Delay', 'Min Delay2'], inplace=True)
df_2018.rename(columns={'min_delay_combined': 'Min Delay'}, inplace=True)
df_2018.head()

,Report Date,Route,Time,Day,Location,Incident,Min Gap,Direction,Vehicle,Min Delay
0,2018-01-01,165,00:17:00,Monday,Wilson Stn,Mechanical,20.0,W/B,1291.0,10.0
1,2018-01-01,706,00:20:00,Monday,Birchmount Garage,Utilized Off Route,1.0,S/B,8123.0,1.0
2,2018-01-01,28,00:30:00,Monday,Davisville Station,Investigation,60.0,S/B,8306.0,30.0
3,2018-01-01,12,00:55:00,Monday,Kennedy Station,Mechanical,44.0,S/B,7777.0,22.0
4,2018-01-01,72,00:55:00,Monday,Pape Station,Mechanical,36.0,NaN,7787.0,18.0


In [6]:
df_2019 = read_excel_sheets('bus_2019.xlsx') 

In [7]:
df_2019['min_delay_combined'] = df_2019['Min Delay'].combine_first(df_2019['Delay'])
df_2019.drop(columns=['Min Delay', 'Delay'], inplace=True)
df_2019.rename(columns={'min_delay_combined': 'Min Delay'}, inplace=True)

df_2019['min_gap_combined'] = df_2019['Min Gap'].combine_first(df_2019['Gap'])
df_2019.drop(columns=['Min Gap', 'Gap'], inplace=True)
df_2019.rename(columns={'min_gap_combined': 'Min Gap'}, inplace=True)

df_2019.head()

,Report Date,Route,Time,Day,Location,Incident,Direction,Vehicle,Incident ID,Min Delay,Min Gap
0,2019-01-01,39,00:13:00,Tuesday,NECR,Mechanical,W/B,1794.0,NaN,9.0,18.0
1,2019-01-01,111,00:15:00,Tuesday,Eglington,Mechanical,S/B,8065.0,NaN,15.0,30.0
2,2019-01-01,35,00:18:00,Tuesday,Finch,Mechanical,S/B,3275.0,NaN,9.0,18.0
3,2019-01-01,25,00:30:00,Tuesday,Don Mills Rd/Eglinton Ave E,Mechanical,N/B,8840.0,NaN,9.0,18.0
4,2019-01-01,36,00:40:00,Tuesday,Humberwood,Investigation,E/B,9119.0,NaN,9.0,18.0


In [145]:
df_2020 = read_excel_sheets('bus_2020.xlsx')

In [114]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36151 entries, 0 to 36150
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Report Date  24860 non-null  datetime64[ns]
 1   Route        36073 non-null  float64       
 2   Time         36151 non-null  object        
 3   Day          36151 non-null  object        
 4   Location     36133 non-null  object        
 5   Incident     36151 non-null  object        
 6   Delay        24805 non-null  float64       
 7   Gap          24789 non-null  float64       
 8   Direction    33633 non-null  object        
 9   Vehicle      33892 non-null  float64       
 10  Date         11291 non-null  datetime64[ns]
 11  Min Delay    11291 non-null  float64       
 12  Min Gap      11291 non-null  float64       
dtypes: datetime64[ns](2), float64(6), object(5)
memory usage: 3.6+ MB


In [146]:
df_2020['min_delay_combined'] = df_2020['Delay'].combine_first(df_2020['Min Delay'])
df_2020.drop(columns=['Min Delay', 'Delay'], inplace=True)
df_2020.rename(columns={'min_delay_combined': 'Min Delay'}, inplace=True)

df_2020['min_gap_combined'] = df_2020['Gap'].combine_first(df_2020['Min Gap'])
df_2020.drop(columns=['Min Gap', 'Gap'], inplace=True)
df_2020.rename(columns={'min_gap_combined': 'Min Gap'}, inplace=True)

df_2020['Date_combined'] = df_2020['Report Date'].combine_first(df_2020['Date'])
df_2020.drop(columns=['Report Date', 'Date'], inplace=True)
df_2020.rename(columns={'Date_combined': 'Report Date'}, inplace=True)

df_2020.head()

,Route,Time,Day,Location,Incident,Direction,Vehicle,Min Delay,Min Gap,Report Date
0,94.0,00:01:00,Wednesday,Ossington station and Castle Frank Station,General Delay,B/W,NaN,10.0,20.0,2020-01-01
1,320.0,01:31:00,Wednesday,Yonge @ Queen,Diversion,B/W,NaN,1.0,1.0,2020-01-01
2,29.0,01:37:00,Wednesday,Dufferin loop,Mechanical,N/B,9028.0,8.0,14.0,2020-01-01
3,102.0,01:40:00,Wednesday,Warden station,Mechanical,N/B,8504.0,11.0,22.0,2020-01-01
4,68.0,01:42:00,Wednesday,Warden Station,Mechanical,N/B,8721.0,8.0,23.0,2020-01-01


In [147]:
df_2021 = read_excel_sheets('bus_2021.xlsx')
df_2021.rename(columns={'Date': 'Report Date'}, inplace=True)
df_2021 = df_2021.iloc[:, :-3]

In [148]:
df_2021.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
0,2021-01-01,320,04:26,Friday,YONGE AND STEELES,Emergency Services,0,0,N,1553
1,2021-01-01,63,05:04,Friday,CANNIFF AND STRACHAN,Mechanical,10,0,N,8144
2,2021-01-01,42,05:34,Friday,WILSON GARAGE,Mechanical,24,48,N,8365
3,2021-01-01,320,05:44,Friday,YONGE AND STEELES,Emergency Services,15,30,N,8181
4,2021-01-01,35,06:21,Friday,JANE STATION,Vision,30,60,E,3186


In [149]:
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42269 entries, 0 to 42268
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Report Date  42269 non-null  datetime64[ns]
 1   Route        39738 non-null  object        
 2   Time         42269 non-null  object        
 3   Day          42269 non-null  object        
 4   Location     42269 non-null  object        
 5   Incident     42269 non-null  object        
 6   Min Delay    42269 non-null  int64         
 7   Min Gap      42269 non-null  int64         
 8   Direction    32952 non-null  object        
 9   Vehicle      42269 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 3.2+ MB


In [150]:
df_2022 = read_excel_sheets('bus_2022.xlsx')
df_2022.rename(columns={'Date': 'Report Date'}, inplace=True)


In [151]:
df_2022.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
0,2022-01-01,320,02:00,Saturday,YONGE AND DUNDAS,General Delay,0,0,NaN,8531
1,2022-01-01,325,02:00,Saturday,OVERLEA AND THORCLIFFE,Diversion,131,161,W,8658
2,2022-01-01,320,02:00,Saturday,YONGE AND STEELES,Operations - Operator,17,20,S,0
3,2022-01-01,320,02:07,Saturday,YONGE AND STEELES,Operations - Operator,4,11,S,0
4,2022-01-01,320,02:13,Saturday,YONGE AND STEELES,Operations - Operator,4,8,S,0


In [152]:
#df_2018 = df_2018.iloc[:, :-1]
#df_2019 = read_excel_sheets('bus_2019.xlsx')
#df_2019 = df_2019.iloc[:, :-3]
#df_2020 = read_excel_sheets('bus_2020.xlsx')
#df_2020 = df_2020.iloc[:, :-3]
#df_2020.rename(columns={'Gap': 'Min Gap', 'Delay': 'Min Delay'}, inplace=True)
#df_2021 = read_excel_sheets('bus_2021.xlsx')


#df_2022 = read_excel_sheets('bus_2022.xlsx')


In [153]:
batch1_list = [df_2014,df_2015,df_2016,df_2017,df_2018,df_2019]
batch1 = pd.concat(batch1_list)

batch1['YTime'] = pd.to_datetime(batch1['Time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M')
batch1['DateTime'] = batch1['Report Date'].dt.strftime('%Y-%m-%d')+'-'+ batch1['YTime']
batch1['DateTime'] = pd.to_datetime(batch1['DateTime'])

drop_columns = ['YTime','Time','Report Date','Direction']
batch1 = batch1.drop(drop_columns, axis = 1)



In [154]:
batch1.head()

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,Incident ID,DateTime
0,95,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,NaN,2014-01-01 00:23:00
1,102,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,NaN,2014-01-01 00:55:00
2,54,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,NaN,2014-01-01 01:28:00
3,112,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,NaN,2014-01-01 01:30:00
4,24,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,NaN,2014-01-01 01:37:00


In [166]:
batch2=df_2020.copy()
batch2['Time2'] = pd.to_datetime(batch2['Time'], format='%H:%M:%S', errors='coerce').dt.time
batch2['DateTime2'] = pd.to_datetime(batch2['Report Date']) + pd.to_timedelta(batch2['Time2'].astype(str))

batch2['Time3'] = pd.to_datetime(batch2['Time'], format='%H:%M', errors='coerce').dt.time
batch2['DateTime3'] = pd.to_datetime(batch2['Report Date']) + pd.to_timedelta(batch2['Time3'].astype(str))

batch2['Datetime_combined'] = batch2['DateTime2'].combine_first(batch2['DateTime3'])

# Rename 'Datetime_combined' to 'Datetime'
batch2.rename(columns={'Datetime_combined': 'Datetime'}, inplace=True)

# Drop unnecessary columns
batch2.drop(['Time2', 'Time', 'DateTime2', 'DateTime3'], axis=1, inplace=True)
batch2.head()

,Route,Day,Location,Incident,Direction,Vehicle,Min Delay,Min Gap,Report Date,Time3,Datetime
0,94.0,Wednesday,Ossington station and Castle Frank Station,General Delay,B/W,NaN,10.0,20.0,2020-01-01,NaT,2020-01-01 00:01:00
1,320.0,Wednesday,Yonge @ Queen,Diversion,B/W,NaN,1.0,1.0,2020-01-01,NaT,2020-01-01 01:31:00
2,29.0,Wednesday,Dufferin loop,Mechanical,N/B,9028.0,8.0,14.0,2020-01-01,NaT,2020-01-01 01:37:00
3,102.0,Wednesday,Warden station,Mechanical,N/B,8504.0,11.0,22.0,2020-01-01,NaT,2020-01-01 01:40:00
4,68.0,Wednesday,Warden Station,Mechanical,N/B,8721.0,8.0,23.0,2020-01-01,NaT,2020-01-01 01:42:00


In [167]:
batch2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36151 entries, 0 to 36150
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Route        36073 non-null  float64       
 1   Day          36151 non-null  object        
 2   Location     36133 non-null  object        
 3   Incident     36151 non-null  object        
 4   Direction    33633 non-null  object        
 5   Vehicle      33892 non-null  float64       
 6   Min Delay    36096 non-null  float64       
 7   Min Gap      36080 non-null  float64       
 8   Report Date  36151 non-null  datetime64[ns]
 9   Time3        11296 non-null  object        
 10  Datetime     36151 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(4), object(5)
memory usage: 3.0+ MB


In [170]:
batch3_list = [df_2021,df_2022]
batch3 = pd.concat(batch3_list)

batch3['Report Date'] = pd.to_datetime(batch3['Report Date'])
batch3['Time'] = pd.to_datetime(batch3['Time'], format='%H:%M').dt.time

# Create a new 'DateTime' column by combining 'Report Date' and 'Time'
batch3['DateTime'] = batch3.apply(
    lambda row: pd.to_datetime(row['Report Date'].replace(hour=row['Time'].hour, minute=row['Time'].minute, second=row['Time'].second)), axis=1
)

drop_columns = ['Time','Report Date','Direction']
batch3 = batch3.drop(drop_columns, axis = 1)
batch3 = batch3.dropna(subset=['Route'])
batch3.head()

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime
0,320,Friday,YONGE AND STEELES,Emergency Services,0,0,1553,2021-01-01 04:26:00
1,63,Friday,CANNIFF AND STRACHAN,Mechanical,10,0,8144,2021-01-01 05:04:00
2,42,Friday,WILSON GARAGE,Mechanical,24,48,8365,2021-01-01 05:34:00
3,320,Friday,YONGE AND STEELES,Emergency Services,15,30,8181,2021-01-01 05:44:00
4,35,Friday,JANE STATION,Vision,30,60,3186,2021-01-01 06:21:00


In [171]:
batch3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98061 entries, 0 to 58706
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Route      98061 non-null  object        
 1   Day        98061 non-null  object        
 2   Location   98061 non-null  object        
 3   Incident   98061 non-null  object        
 4   Min Delay  98061 non-null  int64         
 5   Min Gap    98061 non-null  int64         
 6   Vehicle    98061 non-null  int64         
 7   DateTime   98061 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 6.7+ MB


In [112]:

batches = [batch1, batch2]
df_bus_delay = pd.concat(batches)

ValueError: time data "2020-01-20 2020-01-20 00:00:00" doesn't match format "%Y-%m-%d %H:%M:%S", at position 2630. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [109]:
batch2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137127 entries, 0 to 58706
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Route        134134 non-null  object 
 1   Time         137127 non-null  object 
 2   Day          137127 non-null  object 
 3   Location     137109 non-null  object 
 4   Incident     137127 non-null  object 
 5   Direction    114812 non-null  object 
 6   Vehicle      134868 non-null  float64
 7   Min Delay    137072 non-null  float64
 8   Min Gap      137056 non-null  float64
 9   Report Date  137127 non-null  object 
 10  DateTime     137127 non-null  object 
dtypes: float64(3), object(8)
memory usage: 12.6+ MB


In [ ]:
batch2['YTime'] = pd.to_datetime(batch2['Time'], errors='coerce')
batch2['YTime'] = batch2['YTime'].dt.time
batch2 = batch2.dropna(subset=['YTime'])

batch2['DateTime'] = batch2['Report Date'].dt.strftime('%Y-%m-%d')+'-'+ batch2['YTime']
batch2['DateTime'] = pd.to_datetime(batch2['DateTime'])




drop_columns = ['YTime','Time','Report Date','Direction']
batch2 = batch2.drop(drop_columns, axis = 1)
batch2.head()

In [61]:
batch2.to_csv('batch2check.csv', index=False)

In [46]:
df_bus_delay.head(10)

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,Incident ID,DateTime
0,95,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,NaN,2014-01-01 00:23:00
1,102,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,NaN,2014-01-01 00:55:00
2,54,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,NaN,2014-01-01 01:28:00
3,112,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,NaN,2014-01-01 01:30:00
4,24,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,NaN,2014-01-01 01:37:00
5,129,Wednesday,Scarborough Town,Mechanical,10.0,20.0,1755.0,NaN,2014-01-01 01:50:00
6,36,Wednesday,humberline at woodlot,Mechanical,16.0,32.0,1322.0,NaN,2014-01-01 02:28:00
7,53,Wednesday,Passmore and Markham,Mechanical,18.0,36.0,7043.0,NaN,2014-01-01 02:30:00
8,36,Wednesday,finch stn,Mechanical,16.0,30.0,7933.0,NaN,2014-01-01 02:33:00
9,320,Wednesday,steele at yonge,Mechanical,7.0,14.0,8187.0,NaN,2014-01-01 02:44:00


In [50]:
df_bus_delay.to_csv('df_bus_delaycheck.csv', index=False)

In [49]:
df_bus_delay['copy'] = df_bus_delay['Location'].str.lower()
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('vp','victoria park')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('b/w','').str.replace('w/b','').str.replace('e/b','').str.replace('s/b','').str.replace('n/b','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('stn','station').str.replace(' and ','&').str.replace('/','').str.replace(' ','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('(','').str.replace(')','')
df_bus_delay['copy'] = df_bus_delay['copy'].str.replace('@','&')
df_bus_delay['copy'] = df_bus_delay['copy'].astype(str)
df_bus_delay = df_bus_delay[~df_bus_delay['copy'].str.isdigit()]
df_bus_delay['copy'] = df_bus_delay['copy'].apply(lambda x: '&'.join(sorted(x.split('&'))) if x.count('&') == 1 else x)

stops = pd.read_csv('stops.txt')
stops['cleaned_stops'] = stops['stop_name'].str.replace('Side', '').str.replace(' ', '')
stops['cleaned_stops'] = stops['stop_name'].str.lower()
stops['cleaned_stops'] = stops['stop_name'].str.replace('(north)', '').str.replace('(west)', '').str.replace('(south)', '').str.replace('(east)', '')
stops['cleaned_stops'] = stops['cleaned_stops'].astype(str).str.replace('\d+', '', regex=True)


df_bus_delay_fuzz = df_bus_delay[~df_bus_delay['copy'].str.contains('route')].copy()
df_bus_delay_fuzz = df_bus_delay_fuzz[~df_bus_delay_fuzz['copy'].isin(stops['cleaned_stops'])]
df_bus_delay_fuzz = df_bus_delay_fuzz[df_bus_delay_fuzz['copy'].str.contains('&')]

unique_values = pd.unique(df_bus_delay_fuzz['copy'])

# Create a new DataFrame with a column of unique values
df_delay_fuzz = pd.DataFrame({
    'fuzz_locations': unique_values
})

df_bus_delay.to_csv('bus_delay_raw.csv')
df_delay_fuzz.to_csv('fuzz_locatiins.csv')



In the section below I am testing the fuzz score function we will be using:

This function takes in a string "location" and compares it to each entry in a list of strings "entry". 
It returns a string corresponding to the "best" match for the input string from the list of strings.
We can adjust the threshold to avoid false positives. Strings with no matches above the threshold will not be modified.

In [68]:
from rapidfuzz import process
def find_closest_match_threshold(row, reference_list, threshold = 40):
    match = process.extractOne(row, reference_list)
    if match[1] < threshold:
        return 'not able to fuzz'
    else:
        return match[0]

In [69]:
row = "yonge&steeles"
reference_list = ['Steeles & Yonge']
match = process.extractOne(row, reference_list)
print(match[1])

47.5


In [70]:
def find_closest_match_rapid(row, reference_list):
    match = process.extractOne(row, reference_list)
    return match[0]

In [71]:
df_delay_fuzz['fuzzed'] = df_delay_fuzz['fuzz_locations'].apply(find_closest_match_threshold, args=(stops['stop_name'],))

num_chunks = 10
chunk_size = len(df_delay_fuzz) // num_chunks

# Save each chunk into a separate CSV file
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i < num_chunks - 1 else len(df_delay_fuzz)
    
    chunk_df = df_delay_fuzz.iloc[start_idx:end_idx]
    
    # Save the chunk to a CSV file with a sequential number
    chunk_df.to_csv(f'df_delay_fuzz_{i + 1}.csv', index=False)

In [25]:
script_directory = os.path.dirname(os.path.abspath('df_delay_fuzz1.csv'))

# Use glob to find all .csv files in the directory containing 'bus_delay_fuzz'
csv_files = glob.glob(os.path.join(script_directory, '*df_delay_fuzz*.csv'))

# Initialize an empty list to store individual DataFrames
dfs = []

# Read and append each DataFrame to the list
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
df_fuzz = pd.concat(dfs, ignore_index=True)

In [26]:
bus_delays = pd.merge(df_bus_delay, df_fuzz, left_on='copy', right_on='fuzz_locations', how='left')

# Create the 'analysis' column and fill it with 'fuzz' if 'location' is not NaN, otherwise 'not fuzzed'
bus_delays['analysis'] = bus_delays['fuzzed'].where(~bus_delays['fuzz_locations'].isna(), bus_delays['copy'])

columns_to_drop = ['fuzz_locations', 'fuzzed','copy']
bus_delays = bus_delays.drop(columns=columns_to_drop)

NameError: name 'df_bus_delay' is not defined

In [80]:
bus_delays.drop(columns=['Incident ID'], inplace=True)
bus_delays.head(50)

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime,analysis
0,95.0,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,2014-01-01 00:23:00,yorkmillsstation
1,102.0,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,2014-01-01 00:55:00,entirerunforroute
2,54.0,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,2014-01-01 01:28:00,Warden Ave at Lawrence Ave East
3,112.0,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,2014-01-01 01:30:00,kiplingstation
4,24.0,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,2014-01-01 01:37:00,Ellesmere Rd at Victoria Park Ave
5,129.0,Wednesday,Scarborough Town,Mechanical,10.0,20.0,1755.0,2014-01-01 01:50:00,scarboroughtown
6,36.0,Wednesday,humberline at woodlot,Mechanical,16.0,32.0,1322.0,2014-01-01 02:28:00,humberlineatwoodlot
7,53.0,Wednesday,Passmore and Markham,Mechanical,18.0,36.0,7043.0,2014-01-01 02:30:00,Markham Rd at Passmore Ave
8,36.0,Wednesday,finch stn,Mechanical,16.0,30.0,7933.0,2014-01-01 02:33:00,finchstation
9,320.0,Wednesday,steele at yonge,Mechanical,7.0,14.0,8187.0,2014-01-01 02:44:00,steeleatyonge


In [85]:
bus_delays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588589 entries, 0 to 588588
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Route      588511 non-null  object        
 1   Day        588589 non-null  object        
 2   Location   587796 non-null  object        
 3   Incident   587654 non-null  object        
 4   Min Delay  588127 non-null  float64       
 5   Min Gap    587647 non-null  float64       
 6   Vehicle    518753 non-null  float64       
 7   DateTime   577298 non-null  datetime64[ns]
 8   analysis   588589 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 40.4+ MB


In [86]:
num_chunks = 10
chunk_size = len(bus_delays) // num_chunks

# Save each chunk into a separate CSV file
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i < num_chunks - 1 else len(bus_delays)
    
    chunk_df = bus_delays.iloc[start_idx:end_idx]
    
    # Save the chunk to a CSV file with a sequential number
    chunk_df.to_csv(f'bus_delays_{i + 1}.csv', index=False)

In [87]:
bus_delays.to_csv('bus_delays_.csv', index=False)

In [88]:
bus_delays.head(50)

,Route,Day,Location,Incident,Min Delay,Min Gap,Vehicle,DateTime,analysis
0,95.0,Wednesday,York Mills station,Mechanical,10.0,20.0,1734.0,2014-01-01 00:23:00,yorkmillsstation
1,102.0,Wednesday,Entire run for route,General Delay,33.0,66.0,8110.0,2014-01-01 00:55:00,entirerunforroute
2,54.0,Wednesday,lawrence and Warden,Mechanical,10.0,20.0,7478.0,2014-01-01 01:28:00,Warden Ave at Lawrence Ave East
3,112.0,Wednesday,Kipling Station,Emergency Services,18.0,36.0,8084.0,2014-01-01 01:30:00,kiplingstation
4,24.0,Wednesday,VP and Ellesmere,Investigation,10.0,20.0,7843.0,2014-01-01 01:37:00,Ellesmere Rd at Victoria Park Ave
5,129.0,Wednesday,Scarborough Town,Mechanical,10.0,20.0,1755.0,2014-01-01 01:50:00,scarboroughtown
6,36.0,Wednesday,humberline at woodlot,Mechanical,16.0,32.0,1322.0,2014-01-01 02:28:00,humberlineatwoodlot
7,53.0,Wednesday,Passmore and Markham,Mechanical,18.0,36.0,7043.0,2014-01-01 02:30:00,Markham Rd at Passmore Ave
8,36.0,Wednesday,finch stn,Mechanical,16.0,30.0,7933.0,2014-01-01 02:33:00,finchstation
9,320.0,Wednesday,steele at yonge,Mechanical,7.0,14.0,8187.0,2014-01-01 02:44:00,steeleatyonge
